In [7]:
import pandas as pd

ADMISSION_DATA_COLS = ["subject_id", "hadm_id", "marital_status", "race"]
PERSONAL_DATA_COLS = ["subject_id", "gender", "anchor_age"]
MESURE_DATA_COLS = ['hadm_id', 'stay_id', 'charttime', "itemid", "valuenum"]

In [ ]:
from CONSTANTS import TEMP_PATH


dfPatient = pd.read_csv(TEMP_PATH / "icu_target.csv", usecols=["hadm_id", "stay_id"])
dfPatient.info()

In [13]:
from CONSTANTS import MIMIC_PATH


dfPatientAdmission = pd.read_csv(MIMIC_PATH / "hosp" / "admissions.csv", usecols=ADMISSION_DATA_COLS)
dfPatientPersonal = pd.read_csv(MIMIC_PATH / "hosp" / "patients.csv", usecols=PERSONAL_DATA_COLS)

dfPatientInfo = pd.merge(dfPatientAdmission, dfPatientPersonal, on="subject_id")
del dfPatientAdmission
del dfPatientPersonal

dfPatientInfo.drop(columns="subject_id", inplace=True)
dfPatientInfo["marital_status"] = dfPatientInfo["marital_status"].astype("category")
dfPatientInfo["race"] = dfPatientInfo["race"].astype("category")
dfPatientInfo["gender"] = dfPatientInfo["gender"].astype("category")

dfPatientInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431231 entries, 0 to 431230
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   hadm_id         431231 non-null  int64   
 1   marital_status  421998 non-null  category
 2   race            431231 non-null  category
 3   gender          431231 non-null  category
 4   anchor_age      431231 non-null  int64   
dtypes: category(3), int64(2)
memory usage: 7.8 MB
